In [ ]:
%%capture
import os
import site
os.sys.path.insert(0, '/home/schirrmr/code/reversible/')
os.sys.path.insert(0, '/home/schirrmr/braindecode/code/braindecode/')
%cd /home/schirrmr/


%load_ext autoreload
%autoreload 2
import numpy as np
import logging
log = logging.getLogger()
log.setLevel('INFO')
import sys
logging.basicConfig(format='%(asctime)s %(levelname)s : %(message)s',
                     level=logging.INFO, stream=sys.stdout)
import matplotlib
from matplotlib import pyplot as plt
from matplotlib import cm
%matplotlib inline
%config InlineBackend.figure_format = 'png'
matplotlib.rcParams['figure.figsize'] = (12.0, 1.0)
matplotlib.rcParams['font.size'] = 14
import seaborn
seaborn.set_style('darkgrid')

from reversible2.sliced import sliced_from_samples

from numpy.random import RandomState

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import copy
import math

import itertools
import torch as th
from braindecode.torch_ext.util import np_to_var, var_to_np
from reversible2.spectral_norm import spectral_norm
from reversible2.conv_spectral_norm import conv_spectral_norm

def display_text(text, fontsize=18):
    fig = plt.figure(figsize=(12,0.1))
    plt.title(text, fontsize=fontsize)
    plt.axis('off')
    display(fig)
    plt.close(fig)

In [ ]:
rng = RandomState(394839483)

In [ ]:
a =  rng.rand(200,2)

plt.figure(figsize=(4,4))
plt.plot(a[:,0], a[:,1], marker='o', ls='')
plt.xlim(-2,2)
plt.ylim(-2,2)

In [ ]:
from torch import nn
from reversible2.blocks import dense_add_block

In [ ]:

from reversible2.util import set_random_seeds
from torch.nn import ConstantPad2d
import torch as th


set_random_seeds(2019011641, True)

feature_model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-3)

In [ ]:
n_epochs = 5001
for i_epoch in range(n_epochs):
    examples = th.rand(200,2)
    out = feature_model(examples)
    loss = th.mean(out * out)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        examples = th.rand(200,2)
        a = var_to_np(examples)
        b = var_to_np(feature_model(examples))
        fig = plt.figure(figsize=(4,4))
        plt.plot(a[:,0], a[:,1], marker='o', ls='')
        plt.plot(b[:,0], b[:,1], marker='o', ls='')
        plt.xlim(-2,2)
        plt.ylim(-2,2)
        display(fig)
        plt.close(fig)

In [ ]:
set_random_seeds(3,False)
examples = th.rand(300,2)
a = var_to_np(examples)
b = var_to_np(feature_model(examples))
fig = plt.figure(figsize=(4,4))
plt.plot(a[:,0], a[:,1], marker='o', ls='')
plt.plot(b[:,0], b[:,1], marker='o', ls='')
plt.xlim(-2,2)
plt.ylim(-2,2)
plt.legend(("Inputs", "Encodings"))
plt.title("Loss: Squared L2 norm")
display(fig)
plt.close(fig)

In [ ]:
n_epochs = 5001
for i_epoch in range(n_epochs):
    examples = th.rand(400,2)
    out = feature_model(examples)
    nll_loss = th.mean(out * out)
    ex_a, ex_b, = th.chunk(examples, 2, dim=0)
    out_a, out_b = th.chunk(out, 2, dim=0)
    in_dists = th.norm(ex_a - ex_b, p=2,dim=-1) # euclidean dist
    out_dists = th.norm(out_a - out_b, p=2,dim=-1)
    dist_loss = th.mean(F.relu(out_dists - in_dists) ** 2)
    loss = nll_loss + dist_loss * 100
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        examples = th.rand(200,2)
        a = var_to_np(examples)
        b = var_to_np(feature_model(examples))
        fig = plt.figure(figsize=(4,4))
        plt.plot(a[:,0], a[:,1], marker='o', ls='')
        plt.plot(b[:,0], b[:,1], marker='o', ls='')
        plt.xlim(-2,2)
        plt.ylim(-2,2)
        display(fig)
        plt.close(fig)

In [ ]:


set_random_seeds(20190529, True)

feature_model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-3)

In [ ]:
n_epochs = 5001
for i_epoch in range(n_epochs):
    examples = th.rand(400,2)
    out = feature_model(examples)
    nll_loss = th.mean(out * out)
    ex_a, ex_b, = th.chunk(examples, 2, dim=0)
    out_a, out_b = th.chunk(out, 2, dim=0)
    in_dists = th.norm(ex_a - ex_b, p=2,dim=-1) # euclidean dist
    out_dists = th.norm(out_a - out_b, p=2,dim=-1)
    dist_loss = th.mean(F.relu(out_dists - in_dists) ** 2)
    loss = nll_loss + dist_loss * 100
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        examples = th.rand(200,2)
        a = var_to_np(examples)
        b = var_to_np(feature_model(examples))
        fig = plt.figure(figsize=(4,4))
        plt.plot(a[:,0], a[:,1], marker='o', ls='')
        plt.plot(b[:,0], b[:,1], marker='o', ls='')
        plt.xlim(-2,2)
        plt.ylim(-2,2)
        display(fig)
        plt.close(fig)

In [ ]:


set_random_seeds(201905291, True)

feature_model = nn.Sequential(
    dense_add_block(2,200),
    dense_add_block(2,200),
)

optimizer = th.optim.Adam(feature_model.parameters(), lr=1e-3)

In [ ]:
n_epochs = 5001
for i_epoch in range(n_epochs):
    examples = th.rand(400,2)
    out = feature_model(examples)
    nll_loss = th.mean(out * out)
    ex_a, ex_b, = th.chunk(examples, 2, dim=0)
    out_a, out_b = th.chunk(out, 2, dim=0)
    in_dists = th.norm(ex_a - ex_b, p=2,dim=-1) # euclidean dist
    out_dists = th.norm(out_a - out_b, p=2,dim=-1)
    dist_loss = th.mean(F.relu(out_dists - in_dists) ** 2)
    loss = nll_loss + dist_loss * 100
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    if i_epoch % (n_epochs // 20) == 0:
        examples = th.rand(200,2)
        a = var_to_np(examples)
        b = var_to_np(feature_model(examples))
        fig = plt.figure(figsize=(4,4))
        plt.plot(a[:,0], a[:,1], marker='o', ls='')
        plt.plot(b[:,0], b[:,1], marker='o', ls='')
        plt.xlim(-2,2)
        plt.ylim(-2,2)
        display(fig)
        plt.close(fig)